# Train your own R1 reasoning model with Unsloth


This tutorial demonstrates how to fine-tune the Llama-3.1 8B large language model (LLM) on AMD ROCm GPUs by leveraging [Unsloth](https://unsloth.ai). DeepSeek’s R1 research revealed an “aha moment” where R1-Zero autonomously learned to allocate more thinking time without human feedback by using Group Relative Policy Optimization (GRPO). The Unsloth team has enhanced the entire GRPO process, making it use 80% less VRAM than Hugging Face and Flash Attention 2 (FA2). This lets you reproduce R1-Zero's achievement on just 7GB of VRAM using Qwen2.5 (1.5B).

## Prepare the training environment

### 1. Install the required libraries

Install the libraries required for this tutorial.

In [1]:
# Install Unsloth from source
!git clone https://github.com/billishyahao/unsloth.git && cd unsloth && git checkout billhe/rocm &&  pip install .
!pip install unsloth_zoo==2025.3.17
# Install ROCm Bitsandbytes from source 
!git clone --recurse https://github.com/ROCm/bitsandbytes && cd bitsandbytes && git checkout rocm_enabled_multi_backend && pip install -r requirements-dev.txt && cmake -DCOMPUTE_BACKEND=hip -S . && make -j  && pip install .
# This notebook is verified under unsloth==2025.3.19 unsloth_zoo==2025.3.17 bitsandbytes==0.43.3.dev0

Cloning into 'unsloth'...
remote: Enumerating objects: 13626, done.
remote: Total 13626 (delta 0), reused 0 (delta 0), pack-reused 13626 (from 1)
Receiving objects: 100% (13626/13626), 7.50 MiB | 35.72 MiB/s, done.
Resolving deltas: 100% (10261/10261), done.
Branch 'billhe/rocm' set up to track remote branch 'billhe/rocm' from 'origin'.
Switched to a new branch 'billhe/rocm'
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/HipMarker-1.0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/rpdTracer-1.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Processing /workspace/unsloth
  Installin

Verify the installation:

In [2]:
# Verify the installation and version of the required libraries
!pip list | grep unsloth

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/HipMarker-1.0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/rpdTracer-1.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
unsloth                           2025.3.19
unsloth_zoo                       2025.3.17


Here is the expected output:

```
unsloth                           2025.3.19
unsloth_zoo                       2025.3.17
```

### 2. Provide your Hugging Face token

You'll require a Hugging Face API token to access Llama-3.1. Generate your token at [Hugging Face Tokens](https://huggingface.co/settings/tokens) and request access for [Llama-3.1 8B](https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct). Tokens typically start with "hf_". 

Run the following interactive block in your Jupyter notebook to set up the token:

**Note**: Uncheck the "Add token as Git credential" option.

In [3]:
from huggingface_hub import notebook_login, HfApi

# Prompt the user to log in
notebook_login()

Check your huggingface login:

In [4]:
from huggingface_hub import HfApi

try:
    api = HfApi()
    user_info = api.whoami()
    print(f"Token validated successfully! Logged in as: {user_info['name']}")
except Exception as e:
    print(f"Token validation failed. Error: {e}")

Token validation failed. Error: Token is required (`token=True`), but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.


## Running GRPO

Follow these steps to prepare the data, train the model, and perform inference.

### Set the parameters

Load `Llama 3.1 8B Instruct` and set the parameters:

In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # True for LoRA 4bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



/usr/local/lib/python3.12/dist-packages/unsloth/__init__.py:154: UserWarning: Unsloth: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/unsloth/__init__.py:188: UserWarning: Unsloth: CUDA is not linked properly.
Try running `python -m bitsandbytes` then `python -m xformers.info`
We tried running `ldconfig /usr/lib64-nvidia` ourselves, but it didn't work.
You need to run in your terminal `sudo ldconfig /usr/lib64-nvidia` yourself, then import Unsloth.
Also try `sudo ldconfig /usr/local/cuda-xx.x` - find the latest cuda version.
Unsloth will still run for now, but maybe it might crash - let's hope it works!
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-21 08:39:28 [__init__.py:256] Automatically detected platform rocm.
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.1. vLLM: 0.7.4.dev388+g51641aaa7.rocm631.
   \\   /|    AMD Instinct MI300X VF. Num GPUs = 1. Max memory: 191.688 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0a0+git6c0e746. CUDA: 9.4. CUDA Toolkit: None. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Meta-Llama-3.1-8B-Instruct with actual GPU utilization = 59.84%
Unsloth: Your GPU has CUDA compute capability 9.4 with VRAM = 191.69 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 400.
Unsloth: vLLM's KV C

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

...

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 02-21 08:40:02 [loader.py:429] Loading weights took 5.47 seconds
INFO 02-21 08:40:02 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-21 08:40:03 [model_runner.py:1151] Model loading took 15.4629 GB and 17.987126 seconds
INFO 02-21 08:40:24 [worker.py:295] Memory profiling takes 20.80 seconds
INFO 02-21 08:40:24 [worker.py:295] the current vLLM instance can use total_gpu_memory (191.69GiB) x gpu_memory_utilization (0.60) = 114.70GiB
INFO 02-21 08:40:24 [worker.py:295] model weights take 15.46GiB; non_torch_memory takes 1.00GiB; PyTorch activation peak memory takes 1.83GiB; the rest of the memory reserved for KV Cache is 96.41GiB.
INFO 02-21 08:40:24 [executor_base.py:111] # rocm blocks: 49363, # CPU blocks: 3072
INFO 02-21 08:40:24 [executor_base.py:116] Maximum concurrency for 1024 tokens per request: 771.30x
INFO 02-21 08:40:25 [model_runner.py:1447] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in

Capturing CUDA graph shapes: 100%|██████████| 53/53 [00:22<00:00,  2.34it/s]

INFO 02-21 08:40:48 [model_runner.py:1575] Graph capturing finished in 23 secs, took 0.35 GiB
INFO 02-21 08:40:48 [llm.py:244] init engine (profile, create kv cache, warmup model) took 45.32 seconds



Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data preparation
<a name="Data"></a>

This tutorial directly leverages [willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data preparation and all reward functions. You are free to create your own method.

In [6]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model

Now set up the GRPO Trainer and all configurations:

In [7]:
max_prompt_length = 256

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


Now you can run the trainer. Scroll up to see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait for 150 to 200 steps to see any action. You probably won't get any reward for the first 100 steps. Please be patient!

| Step | Training loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [8]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 83,886,080/8,114,147,328 (1.03% trained)


-------------------- Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
First, let's find out how many tickets Mr. Benson got a discount on, since he got a discount for every ticket bought that exceeds 10. He bought 12 tickets, so we subtract 10 from 12 to get 2 tickets with a discount.

Now, let's calculate the original cost of 12 tickets:
12 tickets x $40 per ticket = $480

Since each ticket after 10 has a 5% discount, let's find the cost of 1 ticket with a discount. 
First, let's find 5% of $40.
5% of $40 is 0.05 x $40 = $2
So, the cost of 1 ticket with a discount is $40 - $2 = $38.

Now, let's calculate the original cost of 2 tickets with a discount.
2 tickets x $38 per ticket = $76
Now, let's subtract this from the original cost of 12 tickets to get the new cost.
$480 - $76 = $404

</answer>
$404 
Extracted:
First, let's find out how many 

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,-0.000000,-0.167333,0.261200,218.333344,0.000000,-0.167333,0.000000,0.000000,0.000000,0.000000
2,0.000000,-0.298000,0.448346,359.333344,0.000000,-0.298000,0.000000,0.000000,0.000000,0.000000
3,0.000000,-0.212667,0.357294,218.666672,0.000306,-0.212667,0.000000,0.000000,0.000000,0.000000
4,0.000000,1.348000,1.008106,235.166672,0.000371,-0.318667,0.000000,0.000000,0.333333,1.333333
5,0.000000,0.869167,1.108771,166.833344,0.000867,-0.047500,0.000000,0.000000,0.250000,0.666667
6,0.000000,0.747000,1.322256,236.333344,0.000370,-0.169667,0.000000,0.000000,0.250000,0.666667
7,0.000000,0.376000,0.799939,228.166672,0.000418,-0.040667,0.000000,0.000000,0.083333,0.333333
8,0.000000,0.366333,1.010413,211.500000,0.000539,-0.050333,0.000000,0.000000,0.083333,0.333333
9,0.000000,0.957167,1.341983,218.666672,0.000384,-0.292833,0.000000,0.000000,0.250000,1.000000
10,0.000000,0.667333,1.261497,253.666672,0.000305,-0.166000,0.000000,0.000000,0.166667,0.666667


-------------------- Question:
Jane is trying to decide whether to buy a house or a trailer. A house costs $480,000 and a trailer costs $120,000. Each loan will be paid in monthly installments over 20 years. How much more is the monthly payment on the house compared to the trailer? 
Answer:
1500 
Response:
To determine the monthly payment difference, we need to calculate the total cost of ownership for each option and then divide that by the number of months (20 years * 12 months/year). 

First, we calculate the total cost of ownership for each option. 
- The house costs $480,000 and the trailer costs $120,000. 

However, since the house and the trailer have different purchase prices, we must calculate the monthly payments for both. We can use the loan formula: M = P[r(1+r)^n]/[(1+r)^n - 1] where:
- M is the monthly payment
- P is the principal loan amount (purchase price)
- r is the monthly interest rate (we assume this is 0 for this calculation since the problem does not provide inte

TrainOutput(global_step=250, training_loss=0.0006683791250203903, metrics={'train_runtime': 1082.5772, 'train_samples_per_second': 1.386, 'train_steps_per_second': 0.231, 'total_flos': 0.0, 'train_loss': 0.0006683791250203903})

<a name="Inference"></a>
### Inference
Now try the model you just trained. First try the model without any GRPO training:

In [9]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.37s/it, est. speed input: 8.92 toks/s, output: 123.11 toks/s]


'Calculating pi to a high degree of accuracy can be a complex task, but I can provide you with a few methods to estimate pi.\n\n**Method 1: Using the Bailey-Borwein-Plouffe formula**\n\nThis formula is an infinite series that can be used to calculate pi to a high degree of accuracy.\n\nπ = ∑[1/(16^k) * (4/(8k+1) - 2/(8k+4) - 1/(8k+5) - 1/(8k+6))]\n\nWhere k is an integer that starts from 0 and goes to infinity.\n\nHere\'s a Python code snippet that implements this formula:\n\n```python\nimport math\n\ndef calculate_pi(num_terms):\n    pi = 0.0\n    for k in range(num_terms):\n        pi += 1/(16**k) * (4/(8*k+1) - 2/(8*k+4) - 1/(8*k+5) - 1/(8*k+6))\n    return pi\n\nnum_terms = 1000  # Change this value to increase or decrease the accuracy of the calculation\npi = calculate_pi(num_terms)\nprint("Estimated pi:", pi)\n```\n\n**Method 2: Using the Gregory-Leibniz series**\n\nThis formula is another infinite series that can be used to calculate pi.\n\nπ = 4 * (1 - 1/3 + 1/5 - 1/7 + 1/9 - 1

Now try it with the LoRA you just trained with GRPO, but save the LoRA first.

In [10]:
model.save_lora("grpo_saved_lora")

Now you can load the LoRA and test:

In [11]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.07s/it, est. speed input: 10.05 toks/s, output: 80.25 toks/s]


'To calculate pi, we can use the following mathematical formula:\n\nπ ≈ 4 * (1 - 1/3 + 1/5 - 1/7 + 1/9 - 1/11 + ...)\n\nThis is a Leibniz formula for π, which is an infinite series. We can approximate the value of π by adding up a large number of terms.\n\nHere\'s a simple Python code to calculate π using this formula:\n\n```\nimport math\n\ndef calculate_pi(n):\n    pi = 0.0\n    for i in range(n):\n        pi += ((-1) ** i) / (2 * i + 1)\n    pi *= 4\n    return pi\n\n# Number of terms to use in the calculation\nn = 1000000\nprint("Approximate value of π:", calculate_pi(n))\n```\n\nHowever, if we want to use a more precise method, we can use the Bailey–Borwein–Plouffe formula:\n\nπ ≈ Σ (16/(8n+1) - 4/(8n+4) - 2/(8n+5) - 1/(8n+6)) from n=0 to infinity\n\nHere\'s a Python code to calculate π using this formula:\n\n```\ndef calculate_pi(n):\n    pi = 0.0\n    for i in range(n):\n        pi += (16/(8*i+1) - 4/(8*i+4) - 2/(8*i+5) - 1/(8*i+6))\n    return pi\n\n# Number of terms to use in 

The reasoning model is much better. It's not always correct, since you only trained it for an hour or so. It'll be better if you extend the sequence length and train it for longer.

<a name="Save"></a>
### Saving to float16 for VLLM

Unsloth also supports saving to `float16` directly. Select `merged_16bit` for `float16` or `merged_4bit` for `int4`. It also allows `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account. Visit the [Hugging Face token settings](https://huggingface.co/settings/tokens) for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")